In [0]:
import sys
sys.path.append('/home/student/ROI/SparkProgram')
from initspark import *
sc, spark, conf = initspark()

## Homework ##

**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


In [0]:
shippers = spark.read.csv('/home/student/ROI/SparkProgram/datasets/northwind/TSV/shippers', header=False, sep = '\t')
#shippers.show()
print(shippers)
shippers.createOrReplaceTempView('shippers')
print(shippers.collect())
shippers = spark.sql('select cast(_c0 as int) as ShipperID, _c1 as ShipperName, _c2 as Phone from shippers')
shippers.createOrReplaceTempView('shippers')
print(shippers)

def isEven(x):
    return x % 2 == 0

from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark.udf.register('isEven', isEven, BooleanType())

spark.sql('select * FROM Shippers WHERE isEven(ShipperID)').show()


DataFrame[_c0: string, _c1: string, _c2: string]
[Row(_c0='1', _c1='Speedy Express', _c2='(503) 555-9831'), Row(_c0='2', _c1='United Package', _c2='(503) 555-3199'), Row(_c0='3', _c1='Federal Shipping', _c2='(503) 555-9931'), Row(_c0='4', _c1='Alliance Shippers', _c2='1-800-222-0451'), Row(_c0='5', _c1='UPS', _c2='1-800-782-7892'), Row(_c0='6', _c1='DHL', _c2='1-800-225-5345')]
DataFrame[ShipperID: int, ShipperName: string, Phone: string]
+---------+-----------------+--------------+
|ShipperID|      ShipperName|         Phone|
+---------+-----------------+--------------+
|        2|   United Package|(503) 555-3199|
|        4|Alliance Shippers|1-800-222-0451|
|        6|              DHL|1-800-225-5345|
+---------+-----------------+--------------+



In [0]:
o = spark.read.json('Orders_LineItems.json')
o.createOrReplaceTempView('Orders')
sql = '''select OrderId, CustomerID, OrderDate, l.ProductID, l.UnitPrice, l.Quantity, l.Discount
FROM Orders LATERAL VIEW EXPLODE(LineItems) EXPLODED_TABLE AS l
'''
o2 = spark.sql(sql)
o2.show()
o2.createOrReplaceTempView('FlatOrders')
spark.sql('select avg(UnitPrice) from FlatOrders').show()


+-------+----------+--------------------+---------+---------+--------+--------+
|OrderId|CustomerID|           OrderDate|ProductID|UnitPrice|Quantity|Discount|
+-------+----------+--------------------+---------+---------+--------+--------+
|  11074|     SIMOB|1998-05-06T00:00:...|       16|    17.45|      14|    0.05|
|  11075|     RICSU|1998-05-06T00:00:...|       76|     18.0|       2|    0.15|
|  11075|     RICSU|1998-05-06T00:00:...|        2|     19.0|      10|    0.15|
|  11075|     RICSU|1998-05-06T00:00:...|       46|     12.0|      30|    0.15|
|  11076|     BONAP|1998-05-06T00:00:...|       19|      9.2|      10|    0.25|
|  11076|     BONAP|1998-05-06T00:00:...|       14|    23.25|      20|    0.25|
|  11076|     BONAP|1998-05-06T00:00:...|        6|     25.0|      20|    0.25|
|  11077|     RATTC|1998-05-06T00:00:...|       16|    17.45|       2|    0.03|
|  11077|     RATTC|1998-05-06T00:00:...|        8|     40.0|       2|     0.1|
|  11077|     RATTC|1998-05-06T00:00:...